In [ ]:
from pymongo import MongoClient
from pprint import pprint
import requests
import json

In [ ]:
client=MongoClient()

In [ ]:
db=client['test-database']

In [ ]:
db.courses.drop()

In [ ]:
courses = [
    {'title': 'Data Science',
     'lecturer': {
         'name': 'Markus Löcher',
         'department': 'Math',
         'status': 'Professor'
     },
     'semester': 1},
    {'title': 'Data Warehousing',
     'lecturer': {
         'name': 'Roland M. Mueller',
         'department': 'Information Systems',
         'status': 'Professor'
     },
     'semester': 1},
    {'title': 'Business Process Management',
     'lecturer': {
         'name': 'Frank Habermann',
         'department': 'Information Systems',
         'status': 'Professor'
     },
     'semester': 1},
    {'title': 'Stratigic Issues of IT',
     'lecturer': {
         'name': 'Sven Pohland',
         'department': 'Information Systems',
         'status': 'Professor'
     },
     'semester': 1},
    {'title': 'Text, Web and Social Media Analytics Lab',
     'lecturer': {
         'name': 'Markus Löcher',
         'department': 'Math',
         'status': 'Professor'
     },
     'semester': 2},
    {'title': 'Enterprise Architectures for Big Data',
     'lecturer': {
         'name': 'Roland M. Mueller',
         'department': 'Information Systems',
         'status': 'Professor'
     },
     'semester': 2},
    {'title': 'Business Process Integration Lab',
     'lecturer': {
         'name': 'Frank Habermann',
         'department': 'Information Systems',
         'status': 'Professor'
     },
     'semester': 2},
    {'title': 'IT-Security and Privacy',
     'lecturer': {
         'name': 'Dennis Uckel',
         'department': 'Information Systems',
         'status': 'External'
     },
     'semester': 2},
    {'title': 'Research Methods',
     'lecturer': {
         'name': 'Marcus Birkenkrahe',
         'department': 'Information Systems',
         'status': 'Professor'
     },
     'semester': 2},
]

In [ ]:
pprint(courses)

In [ ]:
db.courses.insert_many(courses)

In [ ]:
cursor = db.courses.find()
for document in cursor:
    pprint(document)

In [ ]:
my_json = '{"title": "Master Thesis", "semester": 3}'
another_course = json.loads(my_json)
another_course

In [ ]:
inserted_course = db.courses.insert_one(another_course).inserted_id

In [ ]:
cursor = db.courses.find()
for document in cursor:
    pprint(document)

In [ ]:
result = db.courses.find_one({"title": "Data Science"})
pprint(result)

In [ ]:
print(result["_id"])
print(result["lecturer"]["name"])

In [ ]:
result1 = db.courses.find_one({"semester": 2})
pprint(result1)

In [ ]:
result2 = db.courses.find({"semester": 2})
for document in result2:
    pprint(document['title'])

In [ ]:
result2 = db.courses.find({"semester": 2})
for document in result2:
    pprint(document['lecturer']['name'])

In [ ]:
frank_courses = db.courses.find({"lecturer.name": "Frank Habermann"})
for course in frank_courses:
    pprint(course['title'])
    pprint(course['semester'])

In [ ]:
frank_courses_sem_two = db.courses.find({"lecturer.name": "Frank Habermann", "semester": 2})
for course in frank_courses_sem_two:
    pprint(course['title'])
    pprint(course['semester'])

In [ ]:
frank_markus = db.courses.find({"$or": [{"lecturer.name": "Frank Habermann"}, {"lecturer.name": "Markus Löcher"}]})
for courses in frank_markus:
    pprint(courses['title'] + ' ' + courses['lecturer']['name'])

In [ ]:
not_semester_one = db.courses.find({"semester": {"$gt": 1}})
for course in not_semester_one:
    pprint(course['title'])
    pprint(course['semester'])

In [ ]:
course_count_sem_two = db.courses.count_documents({"semester": 1})
print(course_count_sem_two)

In [ ]:
db = client['nobel']
db.prizes.drop()
db.laureates.drop()

for collection_name in ["prizes", "laureates"]:
    singular=collection_name[:-1]
    response=requests.get('http://api.nobelprize.org/v1/{}.json'.format(singular))
    documents=response.json()[collection_name]
    db[collection_name].insert_many(documents)

In [ ]:
pprint(db.laureates.find_one())

In [ ]:
first_doc = db.laureates.find_one()
first_doc['firstname']

In [ ]:
count_female = db.laureates.count_documents({'gender': 'female'})
print(count_female)

In [ ]:
germany_laureates_count = db.laureates.count_documents({'bornCountry': {"$regex": 'Germany'}})
print(germany_laureates_count)

In [ ]:
german_physics_laureates = db.laureates.find({"bornCountry": {"$regex": "Germany"}, "prizes.category": "physics"})
for laureate in german_physics_laureates:
    pprint(laureate['firstname'])
    pprint(laureate['surname'])
    pprint(laureate['prizes'][-1])

In [ ]:
malala=db.laureates.find_one({"firstname": "Malala"})
pprint(malala)

In [ ]:
female_laureates=db.laureates.find({"gender": "female"}).sort([("prizes.year", 1)])
for laureate in female_laureates:
    print(laureate['firstname'] + ' ' + (laureate['surname']))
    print(laureate['prizes'][0]['year'])